# Create a forge and execute an evolution cycle
In this notebook, we will go through the very few steps needed to run a forge cycle for a given budget. 
Before running this notebook, you must set up a few things:

**Your access to LLM's providers:**

To try Ebiose, you must have access to at least one LLM provider. To provide the credentials, copy-paste the [`model_endpoints_template.yml`](./../model_endpoints_template.yml) file, rename it as `model_endpoints.yml` and fill it with your own credentials.

**Add the root directory to your Python path**

Depending on your settings, you may need to add the root of the repository to your `PYTHONPATH` environment variable. You may also use a `.env` file to do so. copy-paste the [`.env.template`](./../.env.template) file, rename it as `.env` and fill it with your own root directory.

**Use LangFuse for tracing**

Ebiose has chosen LangFuse to provide easy and free observability, through its self-hosted capability. Refer to [Langfuse official documentation](https://langfuse.com/self-hosting) to set it up. Once done, fill `LANGFUSE_SECRET_KEY`, `LANGFUSE_PUBLIC_KEY` and `LANGFUSE_HOST` in the `.env` file.

Other observability tools might be used but are not configured yet.

**Load .env file**

To load the `.env` file, execute:

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Creating a basic forge

In ebiose, a **forge** is where custom agents are created to solve specific problems. The forge is the exclusive origin of new agents. Within each forge, architects agents orchestrate the creation and improvement of agents by reusing existing building blocks from the ecosystem.

To create a forge and run a cycle, you must provide the following:
- a description of the forge, which defines the problem that must be solved by generated agents;
- the expected format of the agent's input and output, defined as Pydantic models;
- an implementation of the `compute_fitness` abstract method that will be used by the forge to evaluate the generated agents.

Let's say we wich to generate agents specialized in solving math problems. The forge description could be:

In [2]:
forge_description = "Solving math word problems"

Next, we need to define the expected input and output formats of the generated agents. These formats are to be defined as Pydantic models. 

For instance, in our context of solving math problems, we want the agent input to be a string which will represent the math problem to be solved and the agent output to be composed of two fields:
- `solution` which will be the final solution to the math problem, given as an integer;
- `rationale` which will be the rationale behind the found solution.

The IO Pydantic models will thus be:

In [3]:
from pydantic import BaseModel

class AgentInput(BaseModel):
        math_problem: str

class AgentOutput(BaseModel):
    solution: int
    rationale: str

Lastly, we must provide a way of evaluating the generated agents through the implementation of the `compute_fitness` abstract method of `AgentForge` class. For the sake of demonstration, we will here return a random float between 0 and 1, so that we don't spend tokens at evaluation.

In [4]:
import random
random.seed(7)

from ebiose.core.agent import Agent
from ebiose.core.agent_forge import AgentForge

class BasicForge(AgentForge):
    async def compute_fitness(self, agent: Agent, compute_token_id: str, **kwargs: dict[str, any]) -> float:
        return random.random()

/Users/xabier/dev/ebiose-core/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


We can now instantiate the forge with the provided elements:

In [5]:
forge = BasicForge(
    name="Basic forge",
    description=forge_description,
    agent_input_model=AgentInput,
    agent_output_model=AgentOutput,
    default_generated_agent_engine_type="langgraph_engine",
    default_model_endpoint_id="azure-gpt-4o-mini"
)

## Running a forge cycle

Once the forge is instantiated, we can start generating agents by running a **forge cycle**. 

To do so, you must define:
- a budget in dollars (the forge cycle will end once this budget is exhausted);
- optionally, a path in which created agents and fitness will be saved accross generations. 

> ⚠️ Note that we need to use `asyncio.run` to launch the forge cycle.

> 🚨 Before executing the following cell, check the amount of budget you have allocated!

> 💡 If you are using VSCode, install the [*Markdown Preview Mermaid Support* extension](https://marketplace.visualstudio.com/items?itemName=bierner.markdown-mermaid) to allow the display of the generated agent's graphs.

In [6]:
import asyncio
import nest_asyncio

from ebiose.core.evo_forging_cycle import EvoForgingCylceConfig
nest_asyncio.apply()

from pathlib import Path
from datetime import UTC, datetime

# the path where results will be saved
current_time = datetime.now(UTC).strftime("%Y-%m-%d_%H-%M-%S")
SAVE_PATH = Path(f"./../data/") / current_time
if not SAVE_PATH.exists():
    SAVE_PATH.mkdir(parents=True)


cycle_config = EvoForgingCylceConfig(
    budget=0.01, # the budget for the forge cycle in dollas
    n_agents_in_population=2, # number of agents in the population, at each generation
    n_selected_agents_from_ecosystem=0, # number of agents selected from the ecosystem at initialization
    n_best_agents_to_return=2, # number of best agents to return at the end of the cycle
    replacement_ratio=0.5, # ratio of agents replaced at each generation
    save_path=SAVE_PATH, # the path where results will be saved (optional)
)

best_agents, best_fitness = asyncio.run(
    forge.run_new_cycle(config=cycle_config)
)


Starting a new cycle for forge Basic forge
****** Initializing agents population ******
Creating 2 new agents with architect agents...
  0%|          | 0/2 [00:00<?, ?it/s]
Initializing structured output agent for model AgentOutput (1)
100%|██████████| 2/2 [00:24<00:00, 12.20s/it]
Agent initialization cost: 0.005875155
Population initialized with 2 agents
Initialization of 2 agents took 0:00:24.439670
Budget left after initialization: 0.004124845 $
****** Running generation 0 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 2045.00it/s]
Agent agent-ef206995-0054-4e56-a257-3fffc2357a63 fitness: 0.45318437637077535, cost: 0.0
Agent agent-946999c7-25b2-4819-bf5b-ddb0e426bb53 fitness: 0.29976699686368236, cost: 0.0
Evaluation took 0:00:00.003985 for a total cost of 0.0 $


# Agent ID: agent-ef206995-0054-4e56-a257-3fffc2357a63
## Fitness: 0.45318437637077535
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Solver_Llm(MathSolverLLM)
	Math_Solver_Llm(MathSolverLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured graph network. Your role is to analyze, interpret, and calculate solutions based on the information provided. You will engage in a series of steps to break down the problem, evaluate your reasoning, and ensure that the solution is accurate and clear. Your output will help guide the next stage of the process. Remember to communicate your thought process clearly and provide all necessary information for the next node to function effectively.
##### MathSolverLLM
You are the MathSolverLLM responsible for interpreting and solving math word problems. Read the following math word problem carefully.

1. Identify the key components of the problem, such as numbers, operations, and relationships between quantities.
2. Break down the problem step by step, explaining your thought process clearly as you go.
3. Perform the necessary calculations to arrive at a solution, and ensure that each step is logical and justified.
4. After arriving at the solution, reflect on your reasoning to confirm the accuracy and validity of your answer. Ask yourself: Did I consider all aspects of the problem? Is my solution reasonable?
5. Finally, present your solution in a clear and concise manner, including any relevant explanations that may help the next stage of the process understand your reasoning.

Make sure to detail every step of your reasoning and provide the final answer in a format that is easy to interpret.

# Agent ID: agent-946999c7-25b2-4819-bf5b-ddb0e426bb53
## Fitness: 0.29976699686368236
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) -->|understood| Llm_Node_2(Solution Generation LLM)
	Llm_Node_1(Problem Understanding LLM) -->|not understood| Llm_Node_1(Problem Understanding LLM)
	Llm_Node_2(Solution Generation LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured process. Each node in this graph has a specific role:

1. The Problem Understanding LLM will analyze the math word problem and extract key components, identifying data and operations required.
2. The Solution Generation LLM will utilize the information provided by the Problem Understanding LLM to perform calculations and derive the solution.
3. The flow of information is directed from the Problem Understanding LLM to the Solution Generation LLM, and finally to the conclusion of the process.

Your task is to communicate effectively with the other nodes while executing your specific role.
##### Problem Understanding LLM
Your role is to carefully analyze the given math word problem and extract its essential components.

1. Read the problem statement thoroughly.
2. Identify the key data points, including numbers, units, and any relevant relationships.
3. Determine the mathematical operations that may be required to solve the problem (e.g., addition, subtraction, multiplication, division).
4. Formulate a clear summary of your understanding of the problem, including the identified data and operations.

After completing your analysis, present your findings in a concise format, ensuring that you include all necessary details that the next LLM will require to generate a solution. Make sure to communicate any ambiguities or uncertainties in the problem.
##### Solution Generation LLM
Your role is to generate a solution based on the information provided by the Problem Understanding LLM.

1. Review the summary provided by the Problem Understanding LLM carefully.
2. Verify that you have all the necessary data points and operations required to solve the problem.
3. Execute the calculations step-by-step, ensuring that each operation is performed correctly.
4. If you encounter any uncertainties or require additional clarification, reflect on the information provided and communicate any questions back to the Problem Understanding LLM.
5. Present the final solution clearly, along with any relevant explanations or justifications for your calculations.

Your output should be comprehensive, allowing for easy understanding of how the solution was derived from the given problem. Ensure that your response is structured logically to facilitate the next steps in the process.



Saving current state to ../data/2025-03-10_10-21-36/generation=0
Starting crossover and mutation...

Initializing structured output agent for model AgentOutput (2)
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:16.862569 for a total cost of 0.0017174850000000001 $
Generation 0 completed in 0:00:16.882985 with a total cost of 0.0017174850000000001 $
Budget left after first generation: 0.0024073600000000007 $
****** Running generation 1 ******
Evaluating current population of 2 agents...
100%|██████████| 2/2 [00:00<00:00, 1834.78it/s]
Agent agent-946999c7-25b2-4819-bf5b-ddb0e426bb53 fitness: 0.03920725704743766, cost: 0.0
Agent agent-3a75311f-f164-4283-87a5-37b2f9388d49 fitness: 0.6682158565343952, cost: 0.0
Evaluation took 0:00:00.009856 for a total cost of 0.0 $


# Agent ID: agent-3a75311f-f164-4283-87a5-37b2f9388d49
## Fitness: 0.6682158565343952
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Solver_Llm(MathSolverLLM)
	Math_Solver_Llm(MathSolverLLM) --> Reasoning_Llm(ReasoningLLM)
	Reasoning_Llm(ReasoningLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured graph network. Your role is to analyze, interpret, and calculate solutions based on the information provided. You will engage in a series of steps to break down the problem, evaluate your reasoning, and ensure that the solution is accurate and clear. Your output will help guide the next stage of the process. Remember to communicate your thought process clearly and provide all necessary information for the next node to function effectively.
##### MathSolverLLM
You are the MathSolverLLM responsible for interpreting and solving math word problems. Read the following math word problem carefully.

1. Identify the key components of the problem, such as numbers, operations, and relationships between quantities.
2. Break down the problem step by step, explaining your thought process clearly as you go.
3. Perform the necessary calculations to arrive at a solution, and ensure that each step is logical and justified.
4. After arriving at the solution, reflect on your reasoning to confirm the accuracy and validity of your answer. Ask yourself: Did I consider all aspects of the problem? Is my solution reasonable?
5. Finally, present your solution in a clear and concise manner, including any relevant explanations that may help the next stage of the process understand your reasoning.

Make sure to detail every step of your reasoning and provide the final answer in a format that is easy to interpret.
##### ReasoningLLM
You are the ReasoningLLM responsible for evaluating the output from the MathSolverLLM. Analyze the provided solution for clarity and correctness. Check if the reasoning is logically consistent and if all components of the problem have been addressed. Provide feedback on areas of improvement, if necessary, and confirm if the solution is valid based on the information presented.

# Agent ID: agent-946999c7-25b2-4819-bf5b-ddb0e426bb53
## Fitness: 0.03920725704743766
```mermaid 
graph LR
	Start_Node[start_node] --> Llm_Node_1(Problem Understanding LLM)
	Llm_Node_1(Problem Understanding LLM) -->|understood| Llm_Node_2(Solution Generation LLM)
	Llm_Node_1(Problem Understanding LLM) -->|not understood| Llm_Node_1(Problem Understanding LLM)
	Llm_Node_2(Solution Generation LLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured process. Each node in this graph has a specific role:

1. The Problem Understanding LLM will analyze the math word problem and extract key components, identifying data and operations required.
2. The Solution Generation LLM will utilize the information provided by the Problem Understanding LLM to perform calculations and derive the solution.
3. The flow of information is directed from the Problem Understanding LLM to the Solution Generation LLM, and finally to the conclusion of the process.

Your task is to communicate effectively with the other nodes while executing your specific role.
##### Problem Understanding LLM
Your role is to carefully analyze the given math word problem and extract its essential components.

1. Read the problem statement thoroughly.
2. Identify the key data points, including numbers, units, and any relevant relationships.
3. Determine the mathematical operations that may be required to solve the problem (e.g., addition, subtraction, multiplication, division).
4. Formulate a clear summary of your understanding of the problem, including the identified data and operations.

After completing your analysis, present your findings in a concise format, ensuring that you include all necessary details that the next LLM will require to generate a solution. Make sure to communicate any ambiguities or uncertainties in the problem.
##### Solution Generation LLM
Your role is to generate a solution based on the information provided by the Problem Understanding LLM.

1. Review the summary provided by the Problem Understanding LLM carefully.
2. Verify that you have all the necessary data points and operations required to solve the problem.
3. Execute the calculations step-by-step, ensuring that each operation is performed correctly.
4. If you encounter any uncertainties or require additional clarification, reflect on the information provided and communicate any questions back to the Problem Understanding LLM.
5. Present the final solution clearly, along with any relevant explanations or justifications for your calculations.

Your output should be comprehensive, allowing for easy understanding of how the solution was derived from the given problem. Ensure that your response is structured logically to facilitate the next steps in the process.



Saving current state to ../data/2025-03-10_10-21-36/generation=1
Starting crossover and mutation...
Number of offsprings: 1/1
Crossover and mutation completed in 0:00:18.284970 for a total cost of 0.001957065 $
Generation 1 completed in 0:00:18.314629 with a total cost of 0.001957065 $
Budget left after new generation: 0.0004502950000000016 $
100%|██████████| 2/2 [00:00<00:00, 3938.31it/s]
Agent agent-3a75311f-f164-4283-87a5-37b2f9388d49 fitness: 0.9930959394666341, cost: 0.0
Agent agent-c1437455-a1ed-4dde-b567-07ee1aaf6d9a fitness: 0.8219247866097149, cost: 0.0
Saving current state to ../data/2025-03-10_10-21-36/generation=2
Cycle completed in 0:00:59.658504 with a total cost of 0.009549705 $
Budget left at final: 0.0004502950000000016 $
Returning 2 best agents


We can now display the best agents that have been returned as follows. Note that:
- all agents can be found in the `SAVE_PATH` directory if you defined one;
- here, the compute fitness only returns a random float, so the following displayed agents have not been truly evaluated. 

Go check [examples/math_forge/math_forge.py](./../examples/math_forge/math_forge.py) to see a fully implemeted forge with a non-random fitness evaluation function.

## Display best agents

In [7]:
forge.display_results(best_agents, best_fitness)

# Agent ID: agent-3a75311f-f164-4283-87a5-37b2f9388d49
## Fitness: 0.9930959394666341
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Solver_Llm(MathSolverLLM)
	Math_Solver_Llm(MathSolverLLM) --> Reasoning_Llm(ReasoningLLM)
	Reasoning_Llm(ReasoningLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured graph network. Your role is to analyze, interpret, and calculate solutions based on the information provided. You will engage in a series of steps to break down the problem, evaluate your reasoning, and ensure that the solution is accurate and clear. Your output will help guide the next stage of the process. Remember to communicate your thought process clearly and provide all necessary information for the next node to function effectively.
##### MathSolverLLM
You are the MathSolverLLM responsible for interpreting and solving math word problems. Read the following math word problem carefully.

1. Identify the key components of the problem, such as numbers, operations, and relationships between quantities.
2. Break down the problem step by step, explaining your thought process clearly as you go.
3. Perform the necessary calculations to arrive at a solution, and ensure that each step is logical and justified.
4. After arriving at the solution, reflect on your reasoning to confirm the accuracy and validity of your answer. Ask yourself: Did I consider all aspects of the problem? Is my solution reasonable?
5. Finally, present your solution in a clear and concise manner, including any relevant explanations that may help the next stage of the process understand your reasoning.

Make sure to detail every step of your reasoning and provide the final answer in a format that is easy to interpret.
##### ReasoningLLM
You are the ReasoningLLM responsible for evaluating the output from the MathSolverLLM. Analyze the provided solution for clarity and correctness. Check if the reasoning is logically consistent and if all components of the problem have been addressed. Provide feedback on areas of improvement, if necessary, and confirm if the solution is valid based on the information presented.

# Agent ID: agent-c1437455-a1ed-4dde-b567-07ee1aaf6d9a
## Fitness: 0.8219247866097149
```mermaid 
graph LR
	Start_Node[start_node] --> Math_Solver_Llm(MathSolverLLM)
	Math_Solver_Llm(MathSolverLLM) --> Reasoning_Llm(ReasoningLLM)
	Reasoning_Llm(ReasoningLLM) --> Verification_Llm(VerificationLLM)
	Verification_Llm(VerificationLLM) --> End_Node[end_node]
 
``` 
## Prompts:
##### Shared context prompt
You are part of a collaborative AI model designed to solve math word problems through a structured graph network. Your role is to analyze, interpret, and calculate solutions based on the information provided. You will engage in a series of steps to break down the problem, evaluate your reasoning, and ensure that the solution is accurate and clear. Your output will help guide the next stage of the process. Remember to communicate your thought process clearly and provide all necessary information for the next node to function effectively.
##### MathSolverLLM
You are the MathSolverLLM responsible for interpreting and solving math word problems. Read the following math word problem carefully.

1. Identify the key components of the problem, such as numbers, operations, and relationships between quantities.
2. Break down the problem step by step, explaining your thought process clearly as you go.
3. Perform the necessary calculations to arrive at a solution, and ensure that each step is logical and justified.
4. After arriving at the solution, reflect on your reasoning to confirm the accuracy and validity of your answer. Ask yourself: Did I consider all aspects of the problem? Is my solution reasonable?
5. Finally, present your solution in a clear and concise manner, including any relevant explanations that may help the next stage of the process understand your reasoning.

Make sure to detail every step of your reasoning and provide the final answer in a format that is easy to interpret.
##### ReasoningLLM
You are the ReasoningLLM responsible for evaluating the output from the MathSolverLLM. Analyze the provided solution for clarity and correctness. Check if the reasoning is logically consistent and if all components of the problem have been addressed. Provide feedback on areas of improvement, if necessary, and confirm if the solution is valid based on the information presented.
##### VerificationLLM
You are the VerificationLLM tasked with ensuring that the solution provided by the ReasoningLLM is comprehensive and correct. Review the solution for completeness, accuracy, and clarity. Confirm that all relevant aspects of the math word problem have been addressed before finalizing the output.

